#Initial Downloads and imports

In [0]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from datetime import datetime
from dateutil.parser import parse

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle, resample

import gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
warnings.simplefilter(action='ignore', category=FutureWarning)
from gensim.models.keyedvectors import KeyedVectors
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.phrases import Phrases, Phraser

import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')
from nltk import word_tokenize

import sys
import csv
csv.field_size_limit(sys.maxsize)

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Lambda, Input, concatenate
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D

1.14.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
# Mount drive to import necessary files:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# def prepare_text_data(titles, content):
#   """ Method to load in data and prepare final pandas.DataFrame for inputting
#       machine learning algorithms

#   Parameters
#   ----------
#   titles : str
#       File name of numpy array of title document embedding vectors
#   content : str
#       File name of numpy array of content document embedding vectors
#   """
#   # file path for main csv file of all text data
#   path = '/content/drive/My Drive/Colab Notebooks/csvData/title_content.csv'
#   # read in main csv file into pandas.DataFrame
#   df = pd.read_csv(path, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
#   # file path for sentiment and finance data
#   path2 = '/content/drive/My Drive/Colab Notebooks/csvData/sent_fin.csv'
#   # read in csv fiel into pandas.DataFrame
#   sent_fin = pd.read_csv(path2, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
#   # remove useless columns
#   sent_fin.drop('Unnamed: 0', axis=1, inplace=True)
#   df.drop('Unnamed: 0', axis=1, inplace=True)
#   print('initial dataframes loaded')
#   # load in document vectors
#   titleVecs = np.load("/content/drive/My Drive/Colab Notebooks/Word2VecData/" + titles,
#                       allow_pickle=True)
#   contentVecs = np.load("/content/drive/My Drive/Colab Notebooks/Word2VecData/" + content,
#                       allow_pickle=True)
#   print('word embeddings loaded')
#   # load document vectors into data frame
#   df['title_vectors'] = pd.Series(titleVecs.tolist())
#   df['content_vectors'] = pd.Series(contentVecs.tolist())
#   print('word embeddings added to dataframe')
  
#   for index, row in df.iterrows(): # convert vectors to numpy arrays
#     row['title_vectors'] = np.array(row['title_vectors'])
#     row['content_vectors'] = np.array(row['content_vectors'])
#   print('text converted to numpy arrays')
#   # find mean document vector per date
#   result_df = df.groupby('date')['title_vectors'].apply(np.mean).to_frame('Title')
#   result_df2 = df.groupby('date')['content_vectors'].apply(np.mean).to_frame('Content')
#   print('mean of document vectors for each date found')
#   # recombine data frames created above
#   result_df['Content'] = pd.Series(result_df2['Content'])
#   result_df = result_df.reset_index().rename(columns={"date" : "Date"})
#   # merge text data with finance and sentiment data
#   result_df = result_df.merge(sent_fin, how='inner', on='Date')
#   return result_df

def prepare_text_data(titles):
  """ Method to load in data and prepare final pandas.DataFrame for inputting
      machine learning algorithms

  Parameters
  ----------
  titles : str
      File name of numpy array of title document embedding vectors
  content : str
      File name of numpy array of content document embedding vectors
  """
  # file path for main csv file of all text data
  path = '/content/drive/My Drive/Colab Notebooks/MSc Project 2019 Aaron Dougherty csvData/title_content.csv'
  # read in main csv file into pandas.DataFrame
  df = pd.read_csv(path, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
  # file path for sentiment and finance data
  path2 = '/content/drive/My Drive/Colab Notebooks/MSc Project 2019 Aaron Dougherty csvData/sent_fin.csv'
  # read in csv fiel into pandas.DataFrame
  sent_fin = pd.read_csv(path2, encoding='ISO-8859-1', engine='python', error_bad_lines=False)
  # remove useless columns
  sent_fin.drop('Unnamed: 0', axis=1, inplace=True)
  df.drop('content', axis=1, inplace=True)
  df.drop('Unnamed: 0', axis=1, inplace=True)
  print('initial dataframes loaded')
  # load in document vectors
  titleVecs = np.load("/content/drive/My Drive/Colab Notebooks/MSc Project 2019 Aaron Dougherty Word2VecData/" + titles,
                      allow_pickle=True)
  print('word embeddings loaded')
  # load document vectors into data frame
  df['title_vectors'] = pd.Series(titleVecs.tolist())
  print('word embeddings added to dataframe')
  
  for index, row in df.iterrows(): # convert vectors to numpy arrays
    row['title_vectors'] = np.array(row['title_vectors'])
  print('text converted to numpy arrays')
  # find mean document vector per date
  result_df = df.groupby('date')['title_vectors'].apply(np.mean).to_frame('Title')
  print('mean of document vectors for each date found')
  # recombine data frames created above
  result_df = result_df.reset_index().rename(columns={"date" : "Date"})
  # merge text data with finance and sentiment data
  result_df = result_df.merge(sent_fin, how='inner', on='Date')
  return result_df

# def explode_df(df):
#   """ Method to explode document vectors into on column per value for feeding
#       into machine learning algorithms

#   Parameters
#   ----------
#   df : pandas.DataFrame
#       data frame to expand
#   """
#   df1 = df.iloc[:, 0:3]
#   df2 = df.iloc[:, 3:]
#   df2['Date'] = df1['Date']
#   splitTitle = pd.DataFrame(df1['Title'].tolist())
#   splitContent = pd.DataFrame(df1['Content'].tolist())
#   splitTitle['Date'] = pd.Series(df1['Date'])
#   splitContent['Date'] = pd.Series(df1['Date'])
#   final = pd.merge(splitTitle, splitContent, how='inner', on='Date')
#   final = final.merge(df2, how='inner', on='Date')
#   return final

def explode_df(df):
  """ Method to explode document vectors into on column per value for feeding
      into machine learning algorithms

  Parameters
  ----------
  df : pandas.DataFrame
      data frame to expand
  """
  df1 = df.iloc[:, 0:2]
  df2 = df.iloc[:, 2:]
  df2['Date'] = df1['Date']
  splitTitle = pd.DataFrame(df1['Title'].tolist())
  splitTitle['Date'] = pd.Series(df1['Date'])
  final = splitTitle.merge(df2, how='inner', on='Date')
  return final

In [0]:
# prepare final data frame
df = prepare_text_data('wordvec2_titles_doc3.npy')

initial dataframes loaded
word embeddings loaded
word embeddings added to dataframe
text converted to numpy arrays
mean of document vectors for each date found


In [0]:
df = explode_df(df)

#Training/Development/Testing Split

In [0]:
def shuffle_df(df):
  """ Method to shuffle rows of a data frame

  Parameters
  ----------
  df : pandas.DataFrame
      Data frame to shuffle
  """
  index = df.index # record data frame indexes
  df = shuffle(df) # ScikitLearn's shuffle method
  df.index = index # reset indexes
  return df

In [0]:
# split into X and y
X = df.iloc[:, 0:-15] # set to -8 to remove financial data and -1 to include; -15 removes sentiment
X.drop('Date', axis=1, inplace=True)
y = df[['Date', 'Direction']]
# split into training, development and testing sets
X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.2, shuffle=False)
X_dev, X_test, y_dev, y_test = train_test_split(X_other, y_other, test_size=0.5, shuffle=False)
# remove useless columns
y_dev.drop('Date', axis=1, inplace=True)
y_test.drop('Date', axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


##Dealing with class imbalance:

In [0]:
def recombine(X, y):
  """ Method to recombine the X and y

  Parameters
  ----------
  X : pandas.DataFrame
      Data frame containing X variables
  y : pandas.DataFrame
      Data frame containing y variables
  """
  train = X_train
  train['Direction'] = y_train['Direction'] # combine X and y
  return train

def address_class_imbalance(df, minority_classification, majority_classification,
                            minority_target_samples, majority_target_samples):
  """ Method to remove class imbalance within a data set for binary classification

  Parameters
  ----------
  df : pandas.DataFrame
      Data frame to check for and remove class imbalances from
  minority_classification : int
      the minority class
  majority_classification : int
      the majority class
  minority_target_samples: int
      number of samples to upsample to
  majority_target_samples: int
      number of samples to downsample to
  """
  # Split data frame in 2, 1 for each class
  minority_class = df[df.Direction == minority_classification]
  majority_class = df[df.Direction == majority_classification]
 
  # down sample majority and up sample minority the desired amount using helper
  # methods
  downsampled_majority = downsample_majority(majority_class, majority_target_samples)
  upsampled_minority = upsample_minority(minority_class, minority_target_samples)
  
  # combine newly sampled data frames
  new_df = upsampled_minority
  new_df = new_df.append(downsampled_majority, ignore_index=True)

  return new_df

# Upsample helper method
def upsample_minority(minority_class, target_samples):
  """ Method to up sample the minority class

  Parameters
  ----------
  minority_class : int
      the minority class
  target_samples: int
      number of samples to upsample to
  """
  return resample(minority_class, replace = True, n_samples = target_samples)

# Downsample helper method
def downsample_majority(majority_class, target_samples):
  """ Method to down sample the majority class

  Parameters
  ----------
  majority_class : int
      the majority class
  majority_target_samples: int
      number of samples to downsample to
  """
  return resample(majority_class, replace = False, n_samples = target_samples)

def find_y_mid_point(df):
  """ Method to find the mid-point between to class counts

  Parameters
  ----------
  df : pandas.DataFrame
      dataframe to find midpoint in between 2 unbalanced classes
  """
  # count number of rows belonging to each binary class
  count_class_0, count_class_1 = df.Direction.value_counts()
  # calculate mid-point
  mid_point = int(count_class_1 + ((count_class_0 - count_class_1)/2))
  return mid_point

In [0]:
# rebalance classes using mis point (up and down sample classes equally)
train = recombine(X_train, y_train) # recombine training sets
target_sample = find_y_mid_point(train) # determine target sample value
# up and down sample the classes to remove imbalance
train = address_class_imbalance(train, 1, 0, target_sample, target_sample)
train = shuffle_df(train) # shuffle the new training set
# Split training set back into X and y
X_train = train.iloc[:, 0:-1]
y_train = train[['Direction']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [0]:
minority_class = train[train.Direction == 1]
majority_class = train[train.Direction == 0]
print(len(minority_class), len(majority_class))

150 150


#SVM

without sentiment or finance

In [0]:
# Titles + content
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.58      0.70      0.64        20
           1       0.57      0.44      0.50        18

    accuracy                           0.58        38
   macro avg       0.58      0.57      0.57        38
weighted avg       0.58      0.58      0.57        38

[[14  6]
 [10  8]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Titles + content
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.SVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        16
           1       0.58      1.00      0.73        22

    accuracy                           0.58        38
   macro avg       0.29      0.50      0.37        38
weighted avg       0.34      0.58      0.42        38

[[ 0 16]
 [ 0 22]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
# Titles only
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.77      0.74      0.76        23
           1       0.62      0.67      0.65        15

    accuracy                           0.71        38
   macro avg       0.70      0.70      0.70        38
weighted avg       0.71      0.71      0.71        38

[[17  6]
 [ 5 10]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


without finance data but with sentiment

In [0]:
# Titles + content
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.67      0.55      0.60        22
           1       0.50      0.62      0.56        16

    accuracy                           0.58        38
   macro avg       0.58      0.59      0.58        38
weighted avg       0.60      0.58      0.58        38

[[12 10]
 [ 6 10]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
# Titles only
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.80      0.70      0.74        23
           1       0.61      0.73      0.67        15

    accuracy                           0.71        38
   macro avg       0.71      0.71      0.71        38
weighted avg       0.73      0.71      0.71        38

[[16  7]
 [ 4 11]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


with finance data and sentiment

In [0]:
# Content + titles
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.78      0.64      0.70        22
           1       0.60      0.75      0.67        16

    accuracy                           0.68        38
   macro avg       0.69      0.69      0.68        38
weighted avg       0.70      0.68      0.69        38

[[14  8]
 [ 4 12]]


In [0]:
# Titles only
# SVM_classifier = svm.SVC(kernel='sigmoid')
SVM_classifier = svm.LinearSVC()

#Train the model using the training sets
SVM_classifier.fit(X_train, y_train)

#Predict the response for test dataset
y_predSVM = SVM_classifier.predict(X_dev)

print(classification_report(y_dev, y_predSVM))
print(confusion_matrix(y_dev, y_predSVM))

              precision    recall  f1-score   support

           0       0.79      0.65      0.71        23
           1       0.58      0.73      0.65        15

    accuracy                           0.68        38
   macro avg       0.68      0.69      0.68        38
weighted avg       0.71      0.68      0.69        38

[[15  8]
 [ 4 11]]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


#RF

without finance or sentiment

In [0]:
# Titles + Content
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.48      0.75      0.59        20
           1       0.29      0.11      0.16        18

    accuracy                           0.45        38
   macro avg       0.38      0.43      0.37        38
weighted avg       0.39      0.45      0.39        38

[[15  5]
 [16  2]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
# Titles only
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.62      0.78      0.69        23
           1       0.44      0.27      0.33        15

    accuracy                           0.58        38
   macro avg       0.53      0.52      0.51        38
weighted avg       0.55      0.58      0.55        38

[[18  5]
 [11  4]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


without finance but with sentiment

In [0]:
# Titles + Content
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.60      0.68      0.64        22
           1       0.46      0.38      0.41        16

    accuracy                           0.55        38
   macro avg       0.53      0.53      0.53        38
weighted avg       0.54      0.55      0.54        38

[[15  7]
 [10  6]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
# Titles only
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.65      0.87      0.74        23
           1       0.57      0.27      0.36        15

    accuracy                           0.63        38
   macro avg       0.61      0.57      0.55        38
weighted avg       0.62      0.63      0.59        38

[[20  3]
 [11  4]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


with finance data and sentiment

In [0]:
# Titles + content
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.64      0.64      0.64        22
           1       0.50      0.50      0.50        16

    accuracy                           0.58        38
   macro avg       0.57      0.57      0.57        38
weighted avg       0.58      0.58      0.58        38

[[14  8]
 [ 8  8]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [0]:
# Titles only
rndm_fc = RandomForestClassifier()
rndm_fc = rndm_fc.fit(X_train, y_train)
y_predRF = rndm_fc.predict(X_dev)

print(classification_report(y_dev, y_predRF))
print(confusion_matrix(y_dev, y_predRF))

              precision    recall  f1-score   support

           0       0.64      0.70      0.67        23
           1       0.46      0.40      0.43        15

    accuracy                           0.58        38
   macro avg       0.55      0.55      0.55        38
weighted avg       0.57      0.58      0.57        38

[[16  7]
 [ 9  6]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


#GBM

without finance or sentiment

In [0]:
# Titles + content
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.54      0.70      0.61        20
           1       0.50      0.33      0.40        18

    accuracy                           0.53        38
   macro avg       0.52      0.52      0.50        38
weighted avg       0.52      0.53      0.51        38

[[14  6]
 [12  6]]


In [0]:
#Titles only
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.68      0.83      0.75        23
           1       0.60      0.40      0.48        15

    accuracy                           0.66        38
   macro avg       0.64      0.61      0.61        38
weighted avg       0.65      0.66      0.64        38

[[19  4]
 [ 9  6]]


without finance but with sentiment

In [0]:
# Titles + content
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.64      0.64      0.64        22
           1       0.50      0.50      0.50        16

    accuracy                           0.58        38
   macro avg       0.57      0.57      0.57        38
weighted avg       0.58      0.58      0.58        38

[[14  8]
 [ 8  8]]


In [0]:
# Titles only
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.76      0.83      0.79        23
           1       0.69      0.60      0.64        15

    accuracy                           0.74        38
   macro avg       0.73      0.71      0.72        38
weighted avg       0.73      0.74      0.73        38

[[19  4]
 [ 6  9]]


with finance and sentiment

In [0]:
# Titles + content
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.55      0.55      0.55        22
           1       0.38      0.38      0.38        16

    accuracy                           0.47        38
   macro avg       0.46      0.46      0.46        38
weighted avg       0.47      0.47      0.47        38

[[12 10]
 [10  6]]


In [0]:
# Titles only
GBM = GradientBoostingClassifier()
GBM = GBM.fit(X_train, y_train)
y_predGBM = GBM.predict(X_dev)

print(classification_report(y_dev, y_predGBM))
print(confusion_matrix(y_dev, y_predGBM))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.76      0.83      0.79        23
           1       0.69      0.60      0.64        15

    accuracy                           0.74        38
   macro avg       0.73      0.71      0.72        38
weighted avg       0.73      0.74      0.73        38

[[19  4]
 [ 6  9]]


#Neural Network Prep

In [0]:
# define documents:
def create_xtext(x_df, col):
  """ Method to format data for the nerual network

  Parameters
  ----------
  X_df : pandas.DataFrame
      X values
  col:  str
      relevant columns
  """
  arr = []
  for index, row in x_df.iterrows():
    arr.append(x_df[col].loc[index]) # add text data to array row by row
  return np.array(arr)
  
# create list of concatenated vectors
def create_xtext_concat(x_df):
  """ Method to format data for the nerual network

  Parameters
  ----------
  X_df : pandas.DataFrame
      X values
  col:  str
      relevant columns
  """
  arr = []
  for index, row in x_df.iterrows():
    # concatenate title and content vectors on each row
    concatenated_vector = np.concatenate((x_df['Title'].loc[index], x_df['Content'].loc[index]),
                                         axis=None)
    arr.append(concatenated_vector) # add text data to array row by row
  return np.array(arr)

In [0]:
# transform titles and content into array format
train_titles = create_xtext(X_train, 'Title')
train_content = create_xtext(X_train, 'Content')

dev_titles = create_xtext(X_dev, 'Title')
dev_content = create_xtext(X_dev, 'Content')

test_titles = create_xtext(X_test, 'Title')
test_content = create_xtext(X_test, 'Content')

# prepare y variables
train_labels = y_train['Direction'].tolist()
dev_labels = y_dev['Direction'].tolist()
test_labels = y_test['Direction'].tolist()

In [0]:
# convert y variables into one hot vector encodings for categorical crossentropy
y_train__binary_matrix = to_categorical(y_train['Direction'], num_classes = 2)
y_dev__binary_matrix = to_categorical(y_dev['Direction'], num_classes = 2)
y_test__binary_matrix = to_categorical(y_test['Direction'], num_classes = 2)

##Concatenate variables for sequential MLP model version

In [0]:
# transform titles and content into array format
training = create_xtext_concat(X_train)

development = create_xtext_concat(X_dev)

testing = create_xtext_concat(X_test)

# prepare y variables
train_labels = y_train['Direction'].tolist()
dev_labels = y_dev['Direction'].tolist()
test_labels = y_test['Direction'].tolist()

In [0]:
train_titles.shape

(300, 600)

#MLP

##Version1

In [0]:
nb_classes = 2

# Inputs
titles_input = Input(shape=(300,), name='titles_input')
content_input = Input(shape=(300,), name='content_input')
 
# the first branch operates on the first input
titles = Dense(2000, activation="relu")(titles_input)
titles = Dropout(0.5)(titles)
titles = Dense(1280, activation="relu")(titles)
titles = Model(inputs=titles_input, outputs=titles)
 
# the second branch opreates on the second input
content = Dense(2000, activation="relu")(content_input)
content = Dropout(0.5)(content)
content = Dense(1280, activation="relu")(content)
content = Model(inputs=content_input, outputs=content)
 
# combine the output of the two branches
concat = concatenate([titles.output, content.output])
 
# apply a FC layer and then a regression prediction on the
# combined outputs
fc = Dense(1000, activation="relu")(concat)
fc = Dropout(0.5)(fc)
fc = Dense(500, activation="relu")(fc)
fc = Dropout(0.1)(fc)
fc = Dense(2, activation="sigmoid")(fc)
 
# our model will accept the inputs of the two branches and
# then output a single value
model = Model(inputs=[titles.input, content.input], outputs=fc)

In [0]:
print(model.summary())

opt = Adam(lr=0.0001, clipnorm=1., decay=1e-6, amsgrad=True) ## decay=1e-6, 
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.fit([train_titles, train_content], y_train__binary_matrix, batch_size = 5, epochs=15, verbose=1, validation_split=0.15)

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
print("Generating test predictions...")
y_predMLP = model.predict([test_titles, test_content], verbose=1)
y_pred_bool = np.argmax(y_predMLP, axis=1)
print(classification_report(y_test, y_pred_bool))
print(confusion_matrix(y_test, y_pred_bool))

##Version 2

In [0]:
nb_classes = 2

# Deep Dumb MLP (DDMLP):
model = Sequential() # creates linear DNN model which consists of a linear
                     # stack of layers
# first number value is the fixed batch size of the layer
model.add(Dense(1000, input_shape=(600,))) # 2D layer specifies input shape
model.add(Activation('relu')) # activation function of layer
# Applies dropout to input
# randomly setting a fraction/rate of input units to 0 at each update during
# training, which helps prevent overfitting
model.add(Dropout(0.4))
model.add(Dense(500))
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(nb_classes)) # final binary classification layer
model.add(Activation('softmax'))

# configure learning process
# Contains optimizer function, loss function and a list of metrics
print(model.summary())

opt = Adam(lr=0.0001, clipnorm=1., decay=1e-6, amsgrad=True) ## decay=1e-6, 
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

print("training...")
history = model.fit(training, y_train__binary_matrix, batch_size = 12, epochs = 20, verbose=1, validation_split=0.15)

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [0]:
y_predMLP2 = model.predict(testing, verbose=1)
y_pred_bool2 = np.argmax(y_predMLP2, axis=1)

acc = accuracy_score(test_labels, preds)
print('Accuracy: %' acc)
print(classification_report(test_labels, y_pred_bool2))
print(confusion_matrix(test_labels, y_pred_bool2))